In [21]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [22]:
# Defining Classes
classes = [[i/10,(i+1)/10] for i in range(-10,10,1)]

In [26]:
class ControlsDataset(Dataset):
    """Dataset that maps camera images into steering angle"""
    def __init__(self, stack_size = 1 ,img_folder = 'data/', csv_path='data/data.csv', classes=classes):
        #stack of images through time (ex. last 5 images taken)
        self.stack_size = stack_size
        self.img_folder = img_folder
        # index_col = 0 so panda does not choose its own indexing
        self.classes = classes
        self.data_frame = pd.read_csv(csv_path)
            
    def __len__(self):
        return len(self.data_frame) - self.stack_size
    
    def __getitem__(self, idx):
        img_filenames = ["{0}{1:06d}.jpg".format(self.img_folder, i) 
                            for i in range(idx, idx+self.stack_size)]
        imgs = [io.imread(img_filename).transpose((2,0,1)) for img_filename in img_filenames]
        img_stack = np.concatenate(imgs, axis=0)
        
        # use the latest image as the control
        
        control = self.data_frame.iloc[idx+self.stack_size]["Angle"]
        control = np.array([control])
        
        sample = {'image': img_stack, 
                  'control': control}
        
        return sample
    
    def convertTOClass(self):
        def myfunc(x):
            for i in range(len(self.classes)):
                start = self.classes[i][0]
                end = self.classes[i][1]
                if start == -1 and (x >= start and x <= end):
                    return i
                elif x > start and x <= end:
                    return i
        self.data_frame.columns = ['Number', 'Angle']
        self.data_frame['Angle'] = self.data_frame.apply(lambda x: myfunc(x.Angle), axis=1)
        

In [27]:
dataset = ControlsDataset(stack_size)
dataset.data_frame.iloc[1]

Number    1.0
Angle     0.0
Name: 1, dtype: float64

In [28]:
if __name__ == "__main__":
    stack_size = 1
    dataset = ControlsDataset(stack_size)
    dataset.convertTOClass()
    dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, num_workers = 0)
    print(dataset.data_frame.head(10))
    
    for i_batch, sampled_batch in enumerate(dataloader):
        #print(sampled_batch['control'][0])
        print(sampled_batch["control"])
        print("-------------------------")
    
    
    img_stack = dataset[0]['image']
    f, ax = plt.subplots(stack_size, 3, figsize=(3*stack_size,25))
    ax = ax.reshape((stack_size,3))
    for k in range(3*stack_size):
        i,j = k//3, k % 3
        ax[i,j].imshow(img_stack[k],cmap='gray')
    

   Number  Angle
0       0      9
1       1      9
2       2      9
3       3      9
4       4      9
5       5      9
6       6      9
7       7      9
8       8      9
9       9      9
tensor([[4],
        [9],
        [9],
        [9]])
-------------------------
tensor([[ 9],
        [ 9],
        [ 9],
        [14]])
-------------------------
tensor([[5],
        [9],
        [9],
        [9]])
-------------------------
tensor([[9],
        [9],
        [9],
        [5]])
-------------------------
tensor([[ 4],
        [11],
        [ 9],
        [ 9]])
-------------------------
tensor([[7],
        [9],
        [9],
        [9]])
-------------------------
tensor([[ 9],
        [ 7],
        [19],
        [ 9]])
-------------------------
tensor([[9],
        [9],
        [8],
        [9]])
-------------------------
tensor([[ 8],
        [ 9],
        [ 9],
        [11]])
-------------------------
tensor([[8],
        [9],
        [9],
        [9]])
-------------------------
tensor(

tensor([[0],
        [9],
        [9],
        [2]])
-------------------------


KeyboardInterrupt: 